In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import xml.etree.ElementTree as ET
import json
import requests
import urllib
import urllib.request
import datetime
import folium
import warnings
import lightgbm
import xgboost
from tqdm import tqdm

pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

In [2]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

a = ['토마토']
code_dict={
    '전라남도 해남군': '536824B002',
    '제주도 제주시': '063057B009',
    '전라남도 무안군': '534833E001',
    '전남 해남군': '536824B002',
    '경상남도 창녕군': '635821A001',
    '전라남도 진도군': '539823A001',
    '경기도 포천시': '487915A001',
    '경상남도 밀양시': '627911A001',
    '경기도 시흥시': '429843A001',
    '강원도 철원군': '269811A001',
    '충청남도 천안시': '330846A001',
    '경상북도 청도군': '714902A001',
    '부산 강서구': '618803A001',
    '전북 남원시': '590823A001',
    '경상남도 진주시': '660985B001',
    '경상북도 상주시': '742290A001',
    '경상북도 김천시': '037268B004'
}

joosan_dict={
    '배추': '전라남도 해남군',
    '무': '제주도 제주시',
    '양파': '전라남도 무안군',
    '건고추': '전남 해남군',
    '마늘': '경상남도 창녕군',
    '대파': '전라남도 진도군',
    '얼갈이배추': '경기도 포천시',
    '양배추': '제주도 제주시',
    '깻잎': '경상남도 밀양시',
    '시금치': '경기도 포천시',
    '미나리': '경기도 시흥시',
    '당근': '제주도 제주시',
    '파프리카': '강원도 철원군',
    '새송이': '충청남도 천안시',
    '팽이버섯': '경상북도 청도군',
    '토마토': '부산 강서구',
    '청상추': '전북 남원시',
    '백다다기': '충청남도 천안시',
    '애호박': '경상남도 진주시',
    '캠벨얼리': '경상북도 상주시',
    '샤인마스캇': '경상북도 김천시'
    }

In [3]:
weather=pd.read_csv('weather.csv')

In [4]:
def preprocessing1(temp_df,train, pum, len_lag, weather, joosan_dict, code_dict) :
    weather_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    # 품종과 주산지 날씨 mapping
    
    joosanji = joosan_dict[pum]
    joosan_code = code_dict[joosanji]
    joosan_weather = weather[weather['stn_Code']==joosan_code].reset_index(drop=True)
    
    end_index = np.where(joosan_weather['date']=='2020-09-28')[0][0]
    joosan_weather = joosan_weather.iloc[:end_index+1] #2020-09-28 까지만 자르기
    weather_df = weather_df.merge(joosan_weather, on='date', how='left')
    
    # 해당하는 열의 모든 값이 null 값이기 때문에 이열 두개는 삭제 처리  
    weather_df.drop(['condens_Time','gr_Temp'],axis=1,inplace=True)
    # 결측치가 존재함 -> 날씨의 특성상 앞뒤 날씨의 영향을 받거나 혹은 비슷한 시기의 날씨의 특성과 비슷할 것으로 예상
    # 따라서 결측치를 앞날의 데이터를 가져오는 것으로 결정 
    weather_df['sun_Time']=weather_df['sun_Time'].fillna(method='ffill')
    weather_df['soil_Wt']=weather_df['soil_Wt'].fillna(method='ffill')
    wea_col_list=list(weather_df.columns)
    del wea_col_list[0:6]
    #print(weather_df)
    
    # p_lag, q_lag 추가, 날씨 열 추가
    for lag in range(1,len_lag+1) :
      temp_df[f'p_lag_{lag}'] = -1
      temp_df[f'q_lag_{lag}'] = -1
      for col in range(len(wea_col_list)):
        col=wea_col_list[col]
         #or col =="widdir" or col == "wind"
        if(col == "sun_Time" or col == "sun_Qy" or col =="soil_Temp" or col =="soil_Wt" or col == "wind" or col =="widdir"):
            continue
        if(pum == '건고추' or pum == '당근' or pum =='토마토'):
            continue
        temp_df[f'{col}_lag_{lag}'] = -1
      for index in range(lag, len(temp_df)) :
        temp_df.loc[index, f'p_lag_{lag}'] = temp_df[f'{pum}_가격(원/kg)'][index-lag] #1일전, 2일전, ... 가격을 feature로 추가
        temp_df.loc[index, f'q_lag_{lag}'] = temp_df[f'{pum}_거래량(kg)'][index-lag] #1일전, 2일전, ... 거래량을 feature로 추가
        if(pum == '건고추' or pum == '당근' or pum =='토마토'):
            continue
        temp_df.loc[index, f'temp_lag_{lag}'] = weather_df['temp'][index-lag]
        temp_df.loc[index, f'max_Temp_lag_{lag}'] = weather_df['max_Temp'][index-lag]
        temp_df.loc[index, f'min_Temp_lag_{lag}'] = weather_df['min_Temp'][index-lag]
        temp_df.loc[index, f'hum_lag_{lag}'] = weather_df['hum'][index-lag]
        temp_df.loc[index, f'rain_lag_{lag}'] = weather_df['rain'][index-lag]
   
    # month 추가
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    temp_df['week']=temp_df['date'].dt.week
    temp_df['day'] = temp_df['date'].dt.weekday
 
    

    # 예측 대상(1w,2w,4w) 추가
    for week in ['1_week','2_week','4_week'] :
      temp_df[week] = 0
      n_week = int(week[0])
      for index in range(len(temp_df)) :
        try : temp_df[week][index] = temp_df[f'{pum}_가격(원/kg)'][index+7*n_week]
        except : continue

    # 불필요한 column 제거        
    temp_df = temp_df.drop(['date',f'{pum}_거래량(kg)',f'{pum}_가격(원/kg)'], axis=1)
    
    return temp_df

In [5]:
train = pd.read_csv('./train_201512~.csv')
train = train.drop('Unnamed: 0', axis = 1)
train #2015.12 ~ 2020.09

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),...,청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2015-12-01,1288392.0,311.647794,855004.0,397.379895,595884.0,1349.292060,0.0,0.0,23780.0,...,7984.0,6411.0,53780.0,3371.0,72880.0,3677.0,14155.0,2090.0,0.0,0.0
1,2015-12-02,1005712.0,362.469052,687087.0,377.834757,610021.0,1376.753341,0.0,0.0,16160.0,...,7736.0,6777.0,34635.0,3554.0,61924.0,3890.0,14955.0,2091.0,0.0,0.0
2,2015-12-03,1448544.0,363.709311,837533.0,371.056543,757880.0,1287.379532,0.0,0.0,92560.0,...,8040.0,7258.0,64455.0,3574.0,80478.0,3778.0,15969.0,2080.0,0.0,0.0
3,2015-12-04,1065822.0,418.331494,582460.0,347.406088,673494.0,1259.264077,0.0,0.0,15300.0,...,8568.0,8139.0,62537.0,3722.0,85295.0,3824.0,13179.0,2168.0,0.0,0.0
4,2015-12-05,1047453.0,346.322484,617740.0,328.650565,540224.0,1175.858348,0.0,0.0,17974.0,...,9168.0,7946.0,83336.0,3471.0,100114.0,3413.0,12330.0,1928.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1755,2020-09-24,1856965.0,1839.000000,2055640.0,990.000000,2281429.2,990.000000,2818.4,19101.0,134359.9,...,50730.0,4509.0,282212.3,3001.0,313139.7,3426.0,504242.6,3620.0,283196.9,10940.0
1756,2020-09-25,1880095.5,1789.000000,1879261.0,1011.000000,2074513.0,955.000000,1887.1,23095.0,126926.0,...,54322.0,4178.0,312214.8,2999.0,362741.0,3357.0,479683.1,3618.0,303779.6,10844.0
1757,2020-09-26,1661090.9,1760.000000,1709385.7,1075.000000,2089081.2,961.000000,959.0,22510.0,110357.7,...,61213.0,3770.0,327395.8,3065.0,390361.2,3092.0,521493.8,3691.0,313295.7,10636.0
1758,2020-09-27,25396.0,3066.000000,38222.0,1139.000000,18240.0,1056.000000,60.0,22333.0,620.0,...,144.0,4076.0,285.0,3707.0,2464.0,3252.0,21717.0,3567.0,9734.0,10699.0


In [6]:
def make_dataset(data, label, window_size=10):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
scaler3 = MinMaxScaler()
scaler4 = MinMaxScaler()
scaler5 = MinMaxScaler()
scaler6 = MinMaxScaler()
scaler7 = MinMaxScaler()
scaler8 = MinMaxScaler()
scaler9 = MinMaxScaler()
scaler10 = MinMaxScaler()
scaler11= MinMaxScaler()
scaler12 = MinMaxScaler()
scaler13 = MinMaxScaler()
scaler14 = MinMaxScaler()
scaler15 = MinMaxScaler()
scaler16 = MinMaxScaler()
scaler17 = MinMaxScaler()
scaler18 = MinMaxScaler()
scaler19 = MinMaxScaler()
scaler20 = MinMaxScaler()
scaler21 = MinMaxScaler()

In [24]:
model_dict[f'{pum}_model_{week_num}'] = Sequential()
model_dict[f'{pum}_model_{week_num}'].add(LSTM(16, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )
model_dict[f'{pum}_model_{week_num}'].add(Dense(8))
model_dict[f'{pum}_model_{week_num}'].add(Dense(1))

model_path = 'model'
model_dict[f'{pum}_model_{week_num}'].compile(loss='mean_absolute_error', optimizer='SGD')
early_stop = EarlyStopping(monitor='val_loss', patience=100)
filename = os.path.join(model_path, 'tmp_checkpoint1.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

model_dict[f'{pum}_model_{week_num}'].summary()

Model: "sequential_67"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_65 (LSTM)               (None, 16)                1664      
_________________________________________________________________
dense_65 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 9         
Total params: 1,809
Trainable params: 1,809
Non-trainable params: 0
_________________________________________________________________


In [8]:
model_dict = {}
split = 28 #validation

import os

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

for pum in tqdm(unique_pum + unique_kind):
    # 품목 품종별 전처리
    temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    temp_df = preprocessing1(temp_df,train, pum, 28, weather, joosan_dict, code_dict)
    temp_df = temp_df[28:]
    temp_df = temp_df.fillna(method ='ffill')
    temp_df = temp_df.fillna(method ='bfill')
    tmep_df = temp_df.dropna()
   
    train_df = temp_df
    quartile_1 = train_df.quantile(0.25)
    quartile_3 = train_df.quantile(0.75)
    IQR = quartile_3 - quartile_1
    condition =(train_df > (quartile_3 + 1.5 * IQR)) | (train_df < (quartile_1 - 1.5 * IQR))
    condition = condition.any(axis=1)
    search_df = train_df[condition]
    a = train_df.drop(search_df.index, axis=0)
    if(len(a) > 1000):
        temp_df = a  
    
    # 주차별(1,2,4w) 학습
    for week_num in [1,2,4] :
        
        x = temp_df[temp_df[f'{week_num}_week']>0].iloc[:,:-3]
        y = temp_df[temp_df[f'{week_num}_week']>0].iloc[:][f'{week_num}_week']
        
        if(pum == '배추'):
            scaler = scaler1
        if(pum == '무'):
            scaler = scaler2
        if(pum == '양파'):
            scaler = scaler3
        if(pum == '건고추'):
            scaler = scaler4
        if(pum == '마늘'):
            scaler = scaler5
        if(pum == '대파'):
            scaler = scaler6
        if(pum == '얼갈이배추'):
            scaler = scaler7
        if(pum == '양배추'):
            scaler = scaler8
        if(pum == '깻잎'):
            scaler = scaler9
        if(pum == '시금치'):
            scaler = scaler10
        if(pum == '미나리'):
            scaler = scaler11
        if(pum == '당근'):
            scaler = scaler12
        if(pum == '파프리카'):
            scaler = scaler13
        if(pum == '새송이'):
            scaler = scaler14
        if(pum == '팽이버섯'):
            scaler = scaler15
        if(pum == '토마토'):
            print("good")
            scaler = scaler16
        if(pum == '청상추'):
            scaler = scaler17
        if(pum == '백다다기'):
            scaler = scaler18
        if(pum == '애호박'):
            scaler = scaler19
        if(pum == '캠벨얼리'):
            scaler = scaler20
        if(pum == '샤인마스캇'):
            scaler = scaler21
        
        
        df_scaled_x = scaler.fit_transform(x)
        df_scaled_x = pd.DataFrame(df_scaled_x)
        df_scaled_x.columns = temp_df.columns[:-3]
        x = df_scaled_x
    
        feature_cols = x.columns
        label_cols = [f'{week_num}_week']

        # train dataset
        train_feature, train_label = make_dataset(x, y, 1)

        x_train = train_feature[:-split]
        y_train = train_label[:-split]
        x_valid = train_feature[-split:]
        y_valid = train_label[-split:]
        
        model_dict[f'{pum}_model_{week_num}'] = Sequential()
        model_dict[f'{pum}_model_{week_num}'].add(LSTM(16, 
               input_shape=(train_feature.shape[1],train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )
        model_dict[f'{pum}_model_{week_num}'].add(Dense(8))
        model_dict[f'{pum}_model_{week_num}'].add(Dense(1))

        model_path = 'model'
        model_dict[f'{pum}_model_{week_num}'].compile(loss='mean_absolute_error', optimizer='SGD')
        early_stop = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights = True)
        filename = os.path.join(model_path, 'tmp_checkpoint1.h5')
        checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

        history = model_dict[f'{pum}_model_{week_num}'].fit(x_train, y_train, 
                    epochs=10000, 
                    batch_size=16,
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])

  0%|                                                                                           | 0/21 [00:00<?, ?it/s]

Epoch 1/10000
89/89 [==============================] - 1s 3ms/step - loss: 503.1296 - val_loss: 775.4613

Epoch 00001: val_loss improved from inf to 775.46130, saving model to model\tmp_checkpoint1.h5
Epoch 2/10000
89/89 [==============================] - 0s 969us/step - loss: 231.5370 - val_loss: 420.5248

Epoch 00002: val_loss improved from 775.46130 to 420.52481, saving model to model\tmp_checkpoint1.h5
Epoch 3/10000
89/89 [==============================] - 0s 1ms/step - loss: 243.3653 - val_loss: 736.7921

Epoch 00003: val_loss did not improve from 420.52481
Epoch 4/10000
89/89 [==============================] - 0s 1ms/step - loss: 227.1836 - val_loss: 296.5510

Epoch 00004: val_loss improved from 420.52481 to 296.55096, saving model to model\tmp_checkpoint1.h5
Epoch 5/10000
89/89 [==============================] - 0s 1ms/step - loss: 228.7612 - val_loss: 634.7139

Epoch 00005: val_loss did not improve from 296.55096
Epoch 6/10000
89/89 [==============================] - 0s 1ms/ste

89/89 [==============================] - 0s 897us/step - loss: 149.1592 - val_loss: 672.4590

Epoch 00100: val_loss did not improve from 184.49915
Epoch 101/10000
89/89 [==============================] - 0s 1ms/step - loss: 144.5553 - val_loss: 978.3022

Epoch 00101: val_loss did not improve from 184.49915
Epoch 102/10000
89/89 [==============================] - 0s 1ms/step - loss: 147.1146 - val_loss: 394.0817

Epoch 00102: val_loss did not improve from 184.49915
Epoch 103/10000
89/89 [==============================] - 0s 1ms/step - loss: 148.7384 - val_loss: 365.0694

Epoch 00103: val_loss did not improve from 184.49915
Epoch 104/10000
89/89 [==============================] - 0s 1ms/step - loss: 149.8238 - val_loss: 698.5884

Epoch 00104: val_loss did not improve from 184.49915
Epoch 105/10000
89/89 [==============================] - 0s 1ms/step - loss: 145.4406 - val_loss: 193.2218

Epoch 00105: val_loss did not improve from 184.49915
Epoch 106/10000
89/89 [=========================

89/89 [==============================] - 0s 1ms/step - loss: 151.5482 - val_loss: 261.0551

Epoch 00151: val_loss did not improve from 184.49915
Epoch 152/10000
89/89 [==============================] - 0s 1ms/step - loss: 160.7265 - val_loss: 599.5008

Epoch 00152: val_loss did not improve from 184.49915
Epoch 153/10000
89/89 [==============================] - 0s 931us/step - loss: 154.5161 - val_loss: 493.3740

Epoch 00153: val_loss did not improve from 184.49915
Epoch 154/10000
89/89 [==============================] - 0s 945us/step - loss: 157.5494 - val_loss: 362.4919

Epoch 00154: val_loss did not improve from 184.49915
Epoch 155/10000
89/89 [==============================] - 0s 1ms/step - loss: 152.5859 - val_loss: 529.4001

Epoch 00155: val_loss did not improve from 184.49915
Epoch 156/10000
89/89 [==============================] - 0s 1ms/step - loss: 153.1116 - val_loss: 607.6929

Epoch 00156: val_loss did not improve from 184.49915
Epoch 157/10000
89/89 [=======================

89/89 [==============================] - 0s 1ms/step - loss: 210.6626 - val_loss: 353.0190

Epoch 00033: val_loss did not improve from 205.75336
Epoch 34/10000
89/89 [==============================] - 0s 1ms/step - loss: 226.5155 - val_loss: 902.4233

Epoch 00034: val_loss did not improve from 205.75336
Epoch 35/10000
89/89 [==============================] - 0s 897us/step - loss: 222.7641 - val_loss: 758.2857

Epoch 00035: val_loss did not improve from 205.75336
Epoch 36/10000
89/89 [==============================] - 0s 1ms/step - loss: 223.5809 - val_loss: 666.3928

Epoch 00036: val_loss did not improve from 205.75336
Epoch 37/10000
89/89 [==============================] - 0s 1ms/step - loss: 224.0716 - val_loss: 382.6902

Epoch 00037: val_loss did not improve from 205.75336
Epoch 38/10000
89/89 [==============================] - 0s 1ms/step - loss: 227.5399 - val_loss: 244.6245

Epoch 00038: val_loss did not improve from 205.75336
Epoch 39/10000
89/89 [==============================]

...


Epoch 00003: val_loss improved from 12780.17383 to 2014.82458, saving model to model\tmp_checkpoint1.h5
Epoch 4/10000
55/55 [==============================] - 0s 1ms/step - loss: 12951.1963 - val_loss: 12801.0977

Epoch 00004: val_loss did not improve from 2014.82458
Epoch 5/10000
55/55 [==============================] - 0s 1ms/step - loss: 13150.6016 - val_loss: 12800.0342

Epoch 00005: val_loss did not improve from 2014.82458
Epoch 6/10000
55/55 [==============================] - 0s 1ms/step - loss: 13149.2041 - val_loss: 12798.2383

Epoch 00006: val_loss did not improve from 2014.82458
Epoch 7/10000
55/55 [==============================] - 0s 1ms/step - loss: 12990.7197 - val_loss: 12703.0859

Epoch 00007: val_loss did not improve from 2014.82458
Epoch 8/10000
55/55 [==============================] - 0s 1ms/step - loss: 11860.4355 - val_loss: 12792.9609

Epoch 00008: val_loss did not improve from 2014.82458
Epoch 9/10000
55/55 [==============================] - 0s 1ms/step - loss: 

55/55 [==============================] - 0s 1ms/step - loss: 4407.6348 - val_loss: 1616.0863

Epoch 00101: val_loss did not improve from 1551.47974
Epoch 102/10000
55/55 [==============================] - 0s 1ms/step - loss: 4396.2305 - val_loss: 1633.6527

Epoch 00102: val_loss did not improve from 1551.47974
Epoch 103/10000
55/55 [==============================] - 0s 1ms/step - loss: 4398.2314 - val_loss: 1660.6238

Epoch 00103: val_loss did not improve from 1551.47974
Epoch 104/10000
55/55 [==============================] - 0s 1ms/step - loss: 4396.7124 - val_loss: 1649.6039

Epoch 00104: val_loss did not improve from 1551.47974
Epoch 105/10000
55/55 [==============================] - 0s 1ms/step - loss: 4384.6533 - val_loss: 1599.1154

Epoch 00105: val_loss did not improve from 1551.47974
Epoch 106/10000
55/55 [==============================] - 0s 1ms/step - loss: 4397.4102 - val_loss: 1536.3778

Epoch 00106: val_loss improved from 1551.47974 to 1536.37781, saving model to model\tm

55/55 [==============================] - 0s 1ms/step - loss: 4337.3452 - val_loss: 1636.4816

Epoch 00200: val_loss did not improve from 1524.45532
Epoch 201/10000
55/55 [==============================] - 0s 1ms/step - loss: 4338.0747 - val_loss: 1561.0255

Epoch 00201: val_loss did not improve from 1524.45532
Epoch 202/10000
55/55 [==============================] - 0s 1ms/step - loss: 4332.1860 - val_loss: 1539.9163

Epoch 00202: val_loss did not improve from 1524.45532
Epoch 203/10000
55/55 [==============================] - 0s 1ms/step - loss: 4340.5698 - val_loss: 1623.5806

Epoch 00203: val_loss did not improve from 1524.45532
Epoch 204/10000
55/55 [==============================] - 0s 1ms/step - loss: 4338.9717 - val_loss: 1549.1360

Epoch 00204: val_loss did not improve from 1524.45532
Epoch 205/10000
55/55 [==============================] - 0s 1ms/step - loss: 4347.2231 - val_loss: 1585.3158

Epoch 00205: val_loss did not improve from 1524.45532
Epoch 206/10000
55/55 [=========

100%|█████████████████████████████████████████████████████████████████████████████████| 21/21 [37:36<00:00, 107.46s/it]


Epoch 00240: val_loss did not improve from 1524.45532
